# Testing our LRU Cache for Arbitrage Optimization

The biggest bottleneck in our code is finding the optimal arbitrage for our trades. One way we have optimized this is by implementing an LRU cache on the `cycle.optimize()` method. Here, we test that it works.

In [1]:
import time
from src.sim.scenario import Scenario
from src.trades.cycle import _optimize_mem

In [2]:
# Generate markets and prices
scenario = Scenario("baseline")
sample = scenario.pricepaths[0]
scenario.prepare_for_run()  # Set block timestamps
scenario.prepare_for_trades(sample)  # Set External Market Price

[INFO][22:33:54][src.sim.scenario]-833863: Fetching sim_market from subgraph.
[INFO][22:33:56][src.utils.poolgraph]-833863: Found 20 valid cycles of length 3.


In [3]:
# Unpack
cycles = scenario.cycles
cycle = cycles[0]
cycle

Cycle(Trades: [Swap(pool=Curve.fi Factory Plain Pool: crvUSD/TUSD, in=TUSD, out=crvUSD, amt=0), Swap(pool=Curve.fi Factory Plain Pool: crvUSD/USDP, in=crvUSD, out=USDP, amt=0), Swap(pool=External Market (TUSD, USDP), in=Pax Dollar, out=TrueUSD, amt=0)], Expected Profit: None)

In [4]:
# Check single cycle caching
start = time.time()
cycle.optimize()
end = time.time()
print(f"Before caching: {end - start}")
start = time.time()
cycle.optimize()
end = time.time()
print(f"After caching: {end - start}")
_optimize_mem.cache_info()  # Ensure there was one hit and one miss!

Before caching: 0.048227548599243164
After caching: 7.176399230957031e-05


CacheInfo(hits=1, misses=1, maxsize=None, currsize=1)

In [5]:
cycle.execute()
cycle.optimize()
_optimize_mem.cache_info()  # Ensure the cache wasn't hit this time

[INFO][22:33:57][src.trades.cycle]-833863: Executing cycle Cycle(Trades: [Swap(pool=Curve.fi Factory Plain Pool: crvUSD/TUSD, in=TUSD, out=crvUSD, amt=789), Swap(pool=Curve.fi Factory Plain Pool: crvUSD/USDP, in=crvUSD, out=USDP, amt=791), Swap(pool=External Market (TUSD, USDP), in=Pax Dollar, out=TrueUSD, amt=787)], Expected Profit: -1e-18).
[INFO][22:33:57][src.trades.cycle]-833863: Executing trade Swap(pool=Curve.fi Factory Plain Pool: crvUSD/TUSD, in=TUSD, out=crvUSD, amt=789).
[INFO][22:33:57][src.trades.cycle]-833863: Executing trade Swap(pool=Curve.fi Factory Plain Pool: crvUSD/USDP, in=crvUSD, out=USDP, amt=791).
[INFO][22:33:57][src.trades.cycle]-833863: Executing trade Swap(pool=External Market (TUSD, USDP), in=Pax Dollar, out=TrueUSD, amt=787).


CacheInfo(hits=1, misses=2, maxsize=None, currsize=2)

In [6]:
cycle.optimize()
_optimize_mem.cache_info()  # Ensure the cache was hit this time

CacheInfo(hits=2, misses=2, maxsize=None, currsize=2)

In [7]:
sample = scenario.pricepaths[1]
scenario.prepare_for_trades(sample)  # Update External Market Price
cycle.optimize()
_optimize_mem.cache_info()  # Ensure the cache wasn't hit this time because prices changed

CacheInfo(hits=2, misses=3, maxsize=None, currsize=3)

In [8]:
# Nice!